In [1]:
!pip install pyyaml h5py


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
#h5py - bibilioteka potrzebna do zapisu w formacie HDF5

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images, train_labels),  (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [14]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
#użycie callbacks w punktach kontrolnych

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 1.2147 - accuracy: 0.6365
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 21ms/step - loss: 1.1957 - accuracy: 0.6440 - val_loss: 0.7322 - val_accuracy: 0.7830
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4325 - accuracy: 0.8802
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 20ms/step - loss: 0.4335 - accuracy: 0.8780 - val_loss: 0.5069 - val_accuracy: 0.8460
Epoch 3/10
31/32 [============================>.] - ETA: 0s - loss: 0.2758 - accuracy: 0.9345
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.2753 - accuracy: 0.9340 - val_loss: 0.4648 - val_accuracy: 0.8540
Epoch 4/10
31/32 [============================>.] - ETA: 0s - loss: 0.2020 - accuracy: 0.9546
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 16ms/step - lo

In [16]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [21]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'niewytrenowany model -> acc: {100*acc:.2f}%')

32/32 - 0s - loss: 2.3394 - accuracy: 0.1220 - 289ms/epoch - 9ms/step
niewytrenowany model -> acc: 12.20%


In [22]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z wagami -> acc: {100*acc:.2f}%')

32/32 - 0s - loss: 0.4048 - accuracy: 0.8690 - 98ms/epoch - 3ms/step
niewytrenowany model -> acc: 86.90%
